In [1]:
import os
os.environ["PYEPICS_LIBCA"] = os.environ["EPICS_BASE"] + "/lib/" + os.environ["EPICS_HOST_ARCH"] + "/libca.so"

In [2]:
from ophyd import Device, EpicsSignal, EpicsSignalRO, Component
from ophyd.status import Status, SubscriptionStatus


class EpicsSignalWithRBV(EpicsSignal):
    def __init__(self, prefix, **kwargs):
        super().__init__(prefix + "_RBV", write_pv=prefix, **kwargs)


class HE3PSD(Device):
    acquire = Component(EpicsSignalWithRBV, ":Acquire")
    acquire_time = Component(EpicsSignalWithRBV, ":AcquireTime")
    nbins = Component(EpicsSignalWithRBV, ":NBins")

    counts0 = Component(EpicsSignalRO, ":CountsD0")
    counts1 = Component(EpicsSignalRO, ":CountsD1")
    counts2 = Component(EpicsSignalRO, ":CountsD2")
    counts3 = Component(EpicsSignalRO, ":CountsD3")
    counts4 = Component(EpicsSignalRO, ":CountsD4")
    counts5 = Component(EpicsSignalRO, ":CountsD5")
    counts6 = Component(EpicsSignalRO, ":CountsD6")
    counts7 = Component(EpicsSignalRO, ":CountsD7")
    total_counts = Component(EpicsSignalRO, ":TotalCounts")

    live_counts0 = Component(EpicsSignalRO, ":LiveCountsD0")
    live_counts1 = Component(EpicsSignalRO, ":LiveCountsD1")
    live_counts2 = Component(EpicsSignalRO, ":LiveCountsD2")
    live_counts3 = Component(EpicsSignalRO, ":LiveCountsD3")
    live_counts4 = Component(EpicsSignalRO, ":LiveCountsD4")
    live_counts5 = Component(EpicsSignalRO, ":LiveCountsD5")
    live_counts6 = Component(EpicsSignalRO, ":LiveCountsD6")
    live_counts7 = Component(EpicsSignalRO, ":LiveCountsD7")
    live_total_counts = Component(EpicsSignalRO, ":LiveTotalCounts")

    _default_read_attrs = (
        "counts0",
        "counts7",
        "total_counts"
    )

    _default_configuration_attrs = (
        "acquire_time",
        "nbins"
    )

    def trigger(self):
        def check_value(*, old_value, value, **kwargs):
            "Return True when the acquisition is complete, False otherwise."
            return (old_value == 1 and value == 0)

        self.acquire.set(1).wait()
        status = SubscriptionStatus(self.acquire, check_value)
        return status

In [3]:
%matplotlib qt
from bluesky.callbacks.stream import LiveDispatcher
from bluesky.callbacks.core import make_class_safe
from bluesky.callbacks.mpl_plotting import QtAwareCallback
import matplotlib.pyplot as plt


@make_class_safe()
class LiveCounts(QtAwareCallback):
    def __init__(self, fields:list[str], x_axis_in_mm:bool=True, total_length:float=190, x_axis_center:bool=True):
        """Plot the live counts using matplotlib.

        :param fields: List of field names that should be plotted.
        :param x_axis_in_mm: Flag indicating if the x-axis should be in mm, or just the bin index.
        :param total_length: The total length of the detector in mm.
        :param center: Flag indicating if the x-axis should be centered around zero.
        """
        super().__init__()
        self.fields = fields
        self.x_axis_in_mm = x_axis_in_mm
        self.total_length = total_length
        self.x_axis_center = x_axis_center
        
        self.lines = {}
        
    def event(self, doc, **kwargs):
        for field in self.fields:
            if field in doc["data"]:
                data = doc["data"][field]
                nbins = len(data)

                if self.x_axis_in_mm:
                    x_axis = map(lambda x: (x + 0.5) / nbins, range(nbins))
                    x_axis = map(lambda x: x - 0.5, x_axis) if self.x_axis_center else x_axis
                    x_axis = map(lambda x: x * self.total_length, x_axis)
                else:
                    x_axis = range(nbins)
                
                x_axis = list(x_axis)
                self.lines[field].set_data(x_axis, data)

        self.ax.relim(visible_only=True)
        self.ax.autoscale_view(tight=True)
        self.ax.figure.canvas.draw_idle()

    def start(self, doc):
        self.fig, self.ax = plt.subplots()

        self.lines = {field: self.ax.plot([], [], label=str(field))[0] for field in self.fields}
        self.ax.legend(loc="upper left")

        if self.x_axis_in_mm:
            self.ax.set_xlabel("Position [mm]")
        else:
            self.ax.set_xlabel("Bin Index")
        self.ax.set_ylabel("Counts per bin")
            
        self.fig.show()
        super().start(doc)

In [4]:
det = HE3PSD("he3PSD:det1", name="det1")
det.acquire.set(0)
det.acquire_time.set(1).wait()
det.nbins.set(32).wait()

In [5]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import count
from bluesky.preprocessors import monitor_during_wrapper
import databroker

RE = RunEngine()
db = databroker.catalog["he3"]

RE.subscribe(db.v1.insert)
RE.subscribe(LiveTable(["det1_total_counts"]))
RE.subscribe(LiveCounts(["det1_live_counts0", "det1_live_counts7"]))

det.acquire.set(0).wait()
RE(monitor_during_wrapper(count([det], num=1), [det.live_counts0, det.live_counts7]))



+-----------+------------+-------------------+
|   seq_num |       time | det1_total_counts |
+-----------+------------+-------------------+
|         1 | 16:57:54.1 |               354 |
+-----------+------------+-------------------+
generator count ['11cde057'] (scan num: 1)




('11cde057-5f40-439f-aa10-6152aa5fb7d2',)

In [6]:
RE(monitor_during_wrapper(count([det], num=1), [det.live_counts0, det.live_counts7]))



+-----------+------------+-------------------+
|   seq_num |       time | det1_total_counts |
+-----------+------------+-------------------+
|         1 | 16:58:06.2 |               398 |
+-----------+------------+-------------------+
generator count ['b3989c23'] (scan num: 2)




('b3989c23-1de4-46bf-acfb-7ec51655b16c',)

In [7]:
db[-2]

BlueskyRun
  uid='11cde057-5f40-439f-aa10-6152aa5fb7d2'
  exit_status='success'
  2024-04-24 16:57:43.066 -- 2024-04-24 16:57:54.194
  Streams:
    * det1_live_counts7_monitor
    * primary
    * det1_live_counts0_monitor
